# Create RAG DB

Run this notebook to create the vector database about TikTok Ads


In [6]:
from bs4 import SoupStrainer

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

CHROMA_DB_DIRECTORY = 'chroma_db'

In [2]:
links = []
with open('rag_links.txt', 'r') as f:
    for line in f:
        links.append(line.strip())
links[:5]

['https://ads.tiktok.com/help/article/tiktok-ads-structure?lang=en',
 'https://ads.tiktok.com/help/article/campaign-set-up?lang=en',
 'https://ads.tiktok.com/help/article/choose-right-objective?lang=en',
 'https://ads.tiktok.com/help/article/budget?lang=en',
 'https://ads.tiktok.com/help/article/reach-frequency-ad-buying-type?lang=en']

In [3]:
loader = WebBaseLoader(links, bs_kwargs={
    'parse_only': SoupStrainer('div', class_='article_wrapper_slug_content')
})
docs = loader.load()
docs[:5]

[Document(page_content='Ads run on TikTok Ads Manager are structured into 3 parts: "Campaigns", "Ad Groups", and "Ads". Understanding how your campaigns are structured will help you better set up target audiences, allocate your budget and design creative. This will allow you to expand your ad reach, improve your ad performance and ultimately achieve your campaign goals.Below we will go through how these three work together to create a successful advertising campaign.About the Campaign LevelThe first step to running ads on TikTok Ads Manager is to create a "Campaign". This means you need to also determine your campaign objectives (or goals), which are the final results you want your ads to achieve. Here are the basic steps for setting up your campaign:\u200bSelect the campaign objective\u200bSet the budget for the campaign\u200bStart creating an ad groupYou can choose from various objectives like getting more people to install your app (App Install) or driving more people to your site (

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs_split = text_splitter.split_documents(docs)
docs_split[:5]

[Document(page_content='Ads run on TikTok Ads Manager are structured into 3 parts: "Campaigns", "Ad Groups", and "Ads". Understanding how your campaigns are structured will help you better set up target audiences, allocate your budget and design creative. This will allow you to expand your ad reach, improve your ad performance and ultimately achieve your campaign goals.Below we will go through how these three work together to create a successful advertising campaign.About the Campaign LevelThe first step to running ads on TikTok Ads Manager is to create a "Campaign". This means you need to also determine your campaign objectives (or goals), which are the final results you want your ads to achieve. Here are the basic steps for setting up your campaign:\u200bSelect the campaign objective\u200bSet the budget for the campaign\u200bStart creating an ad groupYou can choose from various objectives like getting more people to install your app (App Install) or driving more people to your site (

In [7]:
sbert_embed = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

db = Chroma.from_documents(
    documents=docs_split,
    embedding=sbert_embed,
    collection_name='ads',
    persist_directory=CHROMA_DB_DIRECTORY,
)

/opt/anaconda3/envs/techjam/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
